In [9]:
import pandas as pd
df = pd.read_csv("../preprocessed_data.csv", index_col=0)
df

,/JS,/AA,/OpenAction,/AcroForm,/RichMedia,/Launch,/EmbeddedFile,/XFA,/URI,/F,...,base64,replace,fromCharCode,getAnnots,ip,xref_size,trailer_size,file_size,nof_stream,class
file_name,,,,,,,,,,,,,,,,,,,,,
file_0001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,21.0,21,74.176,5,0
file_0012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,12.0,15,25.765,5,0
file_0018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4,30.0,42,19.368,28,0
file_0022,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,21.0,21,81.480,5,0
file_0023,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,12.0,15,31.237,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
file_2891,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4,0.0,0,23.192,24,1
file_2899,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,9.0,9,11.892,2,1
file_2902,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,6,23.0,23,16.719,19,1


# minmax scaling

In [10]:
from sklearn.preprocessing import MinMaxScaler
import joblib
minmax_scaler = MinMaxScaler()
minmax_scaler.fit(df.iloc[:, :-1])
minmax_scaled = minmax_scaler.fit_transform(df.iloc[:, :-1])
df_scaled = pd.DataFrame(minmax_scaled, index=df.index, columns=df.columns[:-1])
print(list(df_scaled.columns))
df_scaled['class'] = df['class']
joblib.dump(minmax_scaler, "../scaler.save") 
df_scaled.to_csv("../train_scaled.csv")

['/JS', '/AA', '/OpenAction', '/AcroForm', '/RichMedia', '/Launch', '/EmbeddedFile', '/XFA', '/URI', '/F', '/GoToE', '/Extends', '/Annot', '/Length', '/Filter', '/FlateDecode', '/XObject', '/Encrypt', '/CreationDate', '/Author', 'subject', '/ModDate', 'html', 'Script', 'cmd.exe', 'sys', 'root', 'reg', 'exe', 'http', 'domain', 'filepath', 'ascii', 'xml', 'dll', 'unescape', '%u9090%u9090', 'getIcon', 'temptmp', 'eval', '%appdata%', 'chunk', 'base64', 'replace', 'fromCharCode', 'getAnnots', 'ip', 'xref_size', 'trailer_size', 'file_size', 'nof_stream']


In [11]:
minmax_scaler

MinMaxScaler()

# model import
- n_estimators : 학습 모델의 수, 많아질수록 성능 향상의 가능성이 있으나, 속도가 느려짐
- learning_rate : 학습률, 너무 크면 gradient 발산의 가능성이 있으며, 너무 작으면 학습이 느림
- max_depth : 최대 탐색 깊이, 너무 크면 과적합의 가능성, 너무 작으면 학습 성능 저하
- min_samples_split : 분할 종료 최소 샘플 수, 큰 수면 과적합을 막지만 학습 성능 저하 가능성
- min_samples_leaf : leaf node가 되기 위한 최소 샘플 수, min_samples_split과 비슷한 용도
- random_state : 특정 숫자로 고정 시에 실행 시마다 고정된 결과 반환

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=30, learning_rate=0.025, max_depth=4, random_state = 32)

In [ ]:
X = df_scaled.iloc[:, :-1]
y = df_scaled.iloc[:, -1]

In [ ]:
X

In [ ]:
y

In [ ]:
# 모델 학습
model.fit(X, y)

In [12]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X)
accuracy_score(y, y_pred)

0.9809264305177112